In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import torch

sys.path.append('../code')

import toy_data as toy
from vae import VAE

In [ ]:
N = 10000
batch_size = 10

a, b = (np.random.rand(2)+1)/2
print(a, b)
labels = 2*np.pi*(np.random.rand(N, 2)-0.5)

batch_generator = toy.make_batch_generator(labels, [a,b], batch_size)

plt.imshow(batch_generator.__next__()[0][0, 0])

In [ ]:
def plot_mean_variance_imgs():
    average = np.zeros((28, 28))
    dev = np.zeros((28, 28))
    labels = 2*np.pi*(np.random.rand(N, 2)-0.5)
    batch_generator = list(toy.make_batch_generator(labels, [a,b], batch_size))
    for i, (batch_img, _ ) in enumerate(batch_generator):
        average = average + np.sum(batch_img[:, 0], axis=0)
        dev = dev + np.sum(batch_img[:, 0]**2, axis=0)
    average = average/i
    dev = dev/i

    fig, ax = plt.subplots(ncols=2)
    ax[0].imshow(average)
    ax[1].imshow(average**2-dev)

In [ ]:
device = torch.device("cpu")
bottleneck = 10
model = VAE(bottleneck=bottleneck).to(device)
train_loss = model.fit(batch_generator, max_iter=200)

In [ ]:
plt.plot(train_loss)

In [ ]:
with torch.no_grad():
    sample = torch.randn(9, bottleneck).to(device)
    sample = model.decode(sample).cpu()

In [ ]:
from itertools import product
to_ind = np.array(list(product(range(3), range(3))))

fig, ax = plt.subplots(3, 3, sharex=True, sharey=True)
fig.set_size_inches((10, 10))
for i, img in enumerate(np.array(sample)):
    ind = to_ind[i]
    ax[ind[0], ind[1]].imshow(img.reshape(28, 28))
plt.tight_layout()
plt.show()